<a href="https://colab.research.google.com/github/codefordurham/digital-durham/blob/master/DigitalDurhamMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -qq curl g++ make
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz
import os
os.chdir('spatialindex-src-1.8.5')
!./configure
!make
!make install
!pip install rtree
!ldconfig

(Reading database ... 130942 files and directories currently installed.)
Preparing to unpack .../g++_4%3a7.4.0-1ubuntu2.3_amd64.deb ...
Unpacking g++ (4:7.4.0-1ubuntu2.3) over (4:7.4.0-1ubuntu2.2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Setting up g++ (4:7.4.0-1ubuntu2.3) ...
update-alternatives: warning: forcing reinstallation of alternative /usr/bin/g++ because link group c++ is broken
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  556k  100  556k    0     0   849k      0 --:--:-- --:--:-- --:--:--  849k
checking build system type... x86_64-unknown-linux-gnu
checking for g++... g++
checking whether the C++ compiler works... yes
checking for C++ compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C++ 

In [2]:
!pip install rtree
!pip install folium
!pip install geopandas
!pip install pandas
!pip install libspatialindex

import folium
import branca
import geopandas as gpd
from folium.plugins import Search
import rtree
import pandas as pd
from shapely.geometry import Point
import branca.colormap as cm

print(folium.__version__)

     |████████████████████████████████| 901kB 2.7MB/s 
     |████████████████████████████████| 11.2MB 42.7MB/s 
     |████████████████████████████████| 17.9MB 32.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch
  ERROR: Could not find a version that satisfies the requirement libspatialindex (from versions: none)
ERROR: No matching distribution found for libspatialindex
0.8.3


In [0]:
internetLocations = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQj0fn_AhUXrSa41nxhYoSmt8g6Aa_56P-D7OSvMrMkpUseH09a_i1nHva4KByl47sI8H4r31-DMbRY/pub?gid=0&single=true&output=csv")
# internetLocations.tail()

blocks = gpd.read_file('https://github.com/codefordurham/digital-durham/raw/master/DurhamBlocksWithData.geojson', driver='GeoJSON')

accessPercent = gpd.GeoDataFrame(blocks)
# accessPercent.head()

accessPercent['formattedPercent'] = accessPercent['PercWithoutInternet'].map(str) + "%"
# accessPercent.head()

min, max = accessPercent['PercWithoutInternet'].quantile([0.05,0.95]).apply(lambda x: round(x, 2))
mean = round(accessPercent['PercWithoutInternet'].mean(),2)
# print(f"Min: {min}", f"Max: {max}", f"Mean: {mean}", sep="\n\n")

In [25]:
colormap = cm.LinearColormap(
    ['azure', 'khaki', 'orange'], vmin=min, vmax=max)


# branca.colormap.LinearColormap(
#     colors=['#ffffff', '#ffff7c', '#ff7c00', '#7c0000','#5e4542'],
#     index=accessPercent['PercWithoutInternet'].quantile([0.2,0.4,0.6,0.8]),
#     vmin=min,
#     vmax=max
# )

colormap.caption="Percentage of Households without Internet"

colormap

In [0]:
bounds = folium.GeoJson(blocks).get_bounds()
center = [(bounds[0][0] + bounds[1][0]) / 2, (bounds[0][1] + bounds[1][1]) / 2]

m = folium.Map(
  location=center,
  zoom_start=10,
   tiles=None
)

folium.TileLayer (
  tiles='cartodbpositron',
  show=True,
  control=False
).add_to(m)

style_function = lambda x: {
    'fillColor': colormap(x['properties']['PercWithoutInternet']),
    'color': 'black',
    'weight':0.25,
    'fillOpacity':0.65,
    'lineOpacity':0.25
}

durhamBlocks = folium.GeoJson(
    blocks,
    name='Durham Blocks',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME', 'NoIntrnt' , 'Households', 'formattedPercent'],
        aliases=['<div style="width: 140px;">' + item + '</div>' for item in ['Block', 'Households without Internet' , 'Total Households' , 'Percentage of Households without Internet Access']],  
        localize=True,
        style="width:200px;",
        sticky=False
    )
).add_to(m)

In [27]:
def make_point(row):
    return Point(row.lng, row.lat)

def make_marker(row):
  return Markers(row.Label)
  
pointsLib = internetLocations.apply(make_point, axis=1)
libraryLocations = gpd.GeoDataFrame(internetLocations[internetLocations['Location Type'] == "Library"], geometry=pointsLib)
libraryLocations.crs = {'init': 'epsg:4326'}

pointsRest = internetLocations.apply(make_point, axis=1)
restaurantLocations = gpd.GeoDataFrame(internetLocations[internetLocations['Location Type'] == "Restaurant"], geometry=pointsRest)
restaurantLocations.crs = {'init': 'epsg:4326'}
# restaurantLocations.plot()

libraries = folium.FeatureGroup(name='Libraries', control=True)
restaurants = folium.FeatureGroup(name='Restaurants with Wi-fi Access', control=True)

# Red markers are library locations
for index, row in libraryLocations.iterrows():
  folium.CircleMarker([row['lat'], row['lng']], radius=1.5, popup = "<strong>" + row['Location Name']  + "</strong><br/>" + row['LocationConcat'], layer='Libraries', color='red', fill_color='red').add_to(libraries)

# Blue markers are restaurant locations
for index, row in restaurantLocations.iterrows():
  folium.CircleMarker([row['lat'], row['lng']], radius=1.5, layer='Restaurants with Wi-fi Access',color='#1324bc', fill_color='#1324bc', popup = "<strong>" + row['Location Name']  + "</strong><br/>" + row['LocationConcat']).add_to(restaurants)

  
libraries.add_to(m)
restaurants.add_to(m)

In [28]:
colormap.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m